<h2> 1. Import libraries </h2>

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import yfinance as yf

from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import minimize

from Data_Retriever import DataRetriever as DatRet
from Markowitz_PT import MarkowitzPT as MPT
from BHB_Analyzer import BHBAnalyzer as BHBA

--Frequency BHB analysis performed successfully--


<h2> 2. Define experimental variables </h2>

In [2]:
# Define number of trades
trading_n = 10

In [3]:
# Defining stock pool
ticker_df =  pd.DataFrame()
ticker_df["Petroleum"] = ["EQNR.OL", "AKRBP.OL", ]
ticker_df["Seafood (food)"] = ["ORK.OL", "LSG.OL"]

<h2> 3. Retrieve data </h2>

In [4]:
# Retrieve data from yf API: y-m-d
data = DatRet(ticker_df, "2012-10-01", "2025-02-18")

# In function below, set log=True to check for data availability
data.retrieve_data()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


'--Data retrieved successfully--'

<h2> 4. Frequency trading using MPT </h2>

In [12]:
# Generate benchmark weights thorugh MPT using Sharpe ratio
benchmark = MPT(data.returns, 10,trading_n)
benchmark.frequency_optimizing()

--Frequency trading using MPT successfully performed--


<h2> 5. Conduct performance analysis </h2>

In [3]:
import numpy as np

In [ ]:
ben = np.array([0.3, 0.1, 0.4, 0.2])
exp = np.array([0.4, 0.3, 0.2, 0.1])
ret = np.array([0.023, -0.01, -0.017, 0.0023])

In [261]:
n_sec = 2
n_stocks = 2

def analyzer_time_t(ret:list, we:list, wb:list):


    # Returns and weights on sector level for  benchmark
    rb = np.array([wb[i*n_stocks:(i+1)*n_stocks]@ret[int(i*n_stocks):(i+1)*n_stocks] for i in range(n_sec)])
    wb = np.array([sum(wb[i*n_stocks:(i+1)*n_stocks]) for i in range(n_sec)])
    
    # Returns and weights on sector level for portfolio
    re = np.array([we[i*n_stocks:(i+1)*n_stocks]@ret[int(i*n_stocks):(i+1)*n_stocks] for i in range(n_sec)])
    we = np.array([sum(we[i*n_stocks:(i+1)*n_stocks]) for i in range(n_sec)])

    # Total portfolio and benchmark return
    Re = np.dot(we,re)
    Rb = np.dot(wb,rb)

    # Naked allocation and selection effects
    sel_nkd = (1+we*re) / (1+ we*rb) -1
    all_nkd = (1 + (we-wb)*rb) /  (1+(we-wb)*Rb) -1

    # Q value, in three equations for simplicity
    q_top =  np.log(1+Re) - np.log(1+Rb) - sum(np.log((1+sel_nkd)*(1+all_nkd)))
    q_bottom = sum(np.log2(1+sel_nkd)) + sum(np.log2(1+all_nkd))
    q_tot =  q_top/q_bottom
    
    # Optimized selection and allocation effects, on sector level.
    sel_opt = (1+sel_nkd) * np.e**(q_tot*np.log2(1+sel_nkd))
    all_opt = (1+all_nkd) * np.e**(q_tot*np.log2(1+all_nkd))

    # Returns allocation and slection in a list on sector level
    return [all_opt, sel_opt]

In [262]:
analyzer_time_t(ret, exp,  ben )

[array([1.00220325, 1.00146756]), array([1.00020905, 1.00095242])]